In [2]:
!pip install -q requests beautifulsoup4 Flask~=3.0.3 cloudscraper~=1.2.71 pandas~=2.2.2 beautifulsoup4~=4.12.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 7.9 MB/s eta 0:00:00


In [17]:
import cloudscraper
from bs4 import BeautifulSoup
import pandas as pd

class ForexFactoryScraper:
    def __init__(self, impact_filter=False, currency_filter=False):
        url = f'https://www.forexfactory.com/calendar'
        self.df = self.scrape_table(url)
        if impact_filter or currency_filter:
            self.filtered_df = self.filter_dataframe(impact_filter, currency_filter)
        else:
            self.filtered_df = self.df
    def getPageHTML(self,url):       # gets url and returns source html
        scraper = cloudscraper.create_scraper()
        page = scraper.get(url).text
        return page

    def filter_dataframe(self, impact_filter, currency_filter):
        filtered_df = self.df[self.df['Impact'].isin(impact_filter) & self.df['Currency'].isin(currency_filter)]
        return filtered_df

    def scrape_table(self,url, currency_filter=''):
        pageHTML = self.getPageHTML(url)
        soup = BeautifulSoup(pageHTML, 'html.parser')
        table = soup.find('table', class_ = 'calendar__table')
        currencies = table.find_all('td', class_='calendar__cell calendar__currency')
        events = table.find_all('span', class_ = 'calendar__event-title')
        impacts = table.find_all('td', class_="calendar__cell calendar__impact")
        times = table.find_all('td', class_="calendar__cell calendar__time")
        dates = table.find_all('span', class_ = 'date')
        date_splits =str(table).split('<span class="date">')
        del date_splits[0]
        print(len(date_splits))
        # for i, date in enumerate(dates):
        #     nd = html.split(date)
        #     html=nd[1]
        #     date_splits.append(nd[0])

        currencies_list, events_list, impacts_list, times_list, dates_list = [],[],[],[],[]
        for i in range(len(impacts)):
            if 'yel' in str(impacts[i]):
                impacts_list.append('Low')
            elif 'red' in str(impacts[i]):
                impacts_list.append('High')
            elif 'ora' in str(impacts[i]):
                impacts_list.append('Mid')
            elif 'gra' in str(impacts[i]):
                impacts_list.append('None')
            currencies_list.append(currencies[i].text.strip())
            events_list.append(events[i].text.strip())
            times_list.append(times[i].text.strip())
        reset=0
        dates_list=[]
        #print(len(date_splits))
        for i in range(len(events)):
            if events_list[i] in date_splits[reset]:
                dates_list.append(dates[reset].text.strip())
            else:
                reset+=1
        print(len(dates_list))
        if len(dates_list) < len(events_list):
            for i in range(len(events_list)-len(dates_list)):
                dates_list.append(dates[reset].text.strip())
        df = pd.DataFrame({ 'Date': dates_list, 'Time': times_list, 'Impact': impacts_list,'Currency': currencies_list, 'Event': events_list})
        return df

table = ForexFactoryScraper()
table.filtered_df


7
86


,Date,Time,Impact,Currency,Event
0,Sun Jan 5,1:15pm,Low,USD,FOMC Member Daly Speaks
1,Sun Jan 5,8:45pm,Low,CNY,Caixin Services PMI
2,Mon Jan 6,All Day,None,EUR,Italian Bank Holiday
3,Mon Jan 6,All Day,High,EUR,German Prelim CPI m/m
4,Mon Jan 6,2:30am,Low,CHF,Retail Sales y/y
...,...,...,...,...,...
86,Fri Jan 10,,High,USD,Average Hourly Earnings m/m
87,Fri Jan 10,,High,USD,Non-Farm Employment Change
88,Fri Jan 10,,High,USD,Unemployment Rate
89,Fri Jan 10,10:00am,Mid,USD,Prelim UoM Consumer Sentiment


In [ ]:
#